In [ ]:
#boosting algos are the best performing ML algo with high accuracy. 
#All the algos  work on the basis of previous model errors  and avoiding the mistakes  made by previous trained models by weak learners 
#boosting algos are alternative of bagging. 
# it turns the weak lerarners into strong predictions  by focusing on errors made by individual errors.
#instead of agreession, boosted trees learn from the errors during each boosting model. 


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [2]:
#pip install xgboost

In [68]:
from xgboost import XGBRFClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score , classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [69]:
data = pd.read_csv('pima-indians-diabetes.csv')

In [70]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [71]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [72]:
data.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [73]:
data['Outcome'].value_counts() # check wheter dependent data is balanced or not
# if imbalanced then we have to do smoke technique
#usually in binary we do not find imbalanced data


Outcome
0    500
1    268
Name: count, dtype: int64

In [74]:
#split the data
X = data.iloc[:,:-1] # data.drop('Outcome', axis=1)
Y = data.iloc[:, -1] #data.iloc[:,'Outcome']
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state=42)

In [75]:
classifier = XGBRFClassifier()
classifier.fit(X_train,Y_train)
Y_pred = classifier.predict(X_test)

In [76]:
print(confusion_matrix(Y_test, Y_pred))

[[78 21]
 [16 39]]


In [77]:
print(classification_report(Y_test, Y_pred))

              precision    recall  f1-score   support

           0       0.83      0.79      0.81        99
           1       0.65      0.71      0.68        55

    accuracy                           0.76       154
   macro avg       0.74      0.75      0.74       154
weighted avg       0.77      0.76      0.76       154



In [78]:
#apply multi methods and check wheter accuracy has increased or not
#1. CROSSVALIDATION METHOD
accuracies = cross_val_score(estimator=classifier, X= X_train, y= Y_train, cv= 10)
print('Accuracy: {:0.2f}%'.format(accuracies.mean()*100))

Accuracy: 75.72%


In [79]:
classifier.get_params
#Decision trees have very common  problem is overfitting  and XGboost is collection of decision trees.
#when the DT length increases overfitting  increases.

<bound method XGBModel.get_params of XGBRFClassifier(base_score=None, booster=None, callbacks=None,
                colsample_bylevel=None, colsample_bytree=None, device=None,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, feature_types=None, gamma=None,
                grow_policy=None, importance_type=None,
                interaction_constraints=None, max_bin=None,
                max_cat_threshold=None, max_cat_to_onehot=None,
                max_delta_step=None, max_depth=None, max_leaves=None,
                min_child_weight=None, missing=nan, monotone_constraints=None,
                multi_strategy=None, n_estimators=None, n_jobs=None,
                num_parallel_tree=None, objective='binary:logistic',
                random_state=None, reg_alpha=None, ...)>

In [80]:
# higher learning rates requires less boosting rounds. 
# if  you want to restrict your model  and want to increase accuracy, then increase gamma
# max_depth increase the length of tree

In [93]:
#gridsearch will run the model in all combination mentioned in grid and provid the best result. 
parameters = [{'learning_rate': [0.3,0.5,0.7], 'gamma':[0, 0.001, 0.01], 'max_depth':[5,4,6]}]
gridsearchcv = GridSearchCV(estimator=classifier, param_grid=parameters, scoring='accuracy',cv=10, n_jobs=-1  )
# scoring is the evaluation matric
#n_jobs use all avialable cores/ processor to execute the methods
gridsearchcv.fit(X_train, Y_train)

GridSearchCV(cv=10,
             estimator=XGBRFClassifier(base_score=None, booster=None,
                                       callbacks=None, colsample_bylevel=None,
                                       colsample_bytree=None, device=None,
                                       early_stopping_rounds=None,
                                       enable_categorical=False,
                                       eval_metric=None, feature_types=None,
                                       gamma=None, grow_policy=None,
                                       importance_type=None,
                                       interaction_constraints=None,
                                       max_bin=None, max_cat_threshold=None,
                                       ma...
                                       max_delta_step=None, max_depth=None,
                                       max_leaves=None, min_child_weight=None,
                                       missing=nan, monotone_constraints=None,
                                       multi_strategy=None, n_estimators=None,
                                       n_jobs=None, num_parallel_tree=None,
                                       objective='binary:logistic',
                                       random_state=None, reg_alpha=None, ...),
             n_jobs=-1,
             param_grid=[{'gamma': [0, 0.001, 0.01],
                          'learning_rate': [0.3, 0.5, 0.7],
                          'max_depth': [5, 4, 6]}],
             scoring='accuracy')

In [94]:
best_accuracy = gridsearchcv.best_score_
print('best accuracy = {:0.2f}%'.format(best_accuracy*100))
best_parameters = gridsearchcv.best_params_
print('best Parameters = ', best_parameters)

best accuracy = 76.05%
best Parameters =  {'gamma': 0, 'learning_rate': 0.7, 'max_depth': 5}


In [96]:
print(confusion_matrix(Y_test, Y_pred))
print(classification_report(Y_test, Y_pred))

[[78 21]
 [16 39]]
              precision    recall  f1-score   support

           0       0.83      0.79      0.81        99
           1       0.65      0.71      0.68        55

    accuracy                           0.76       154
   macro avg       0.74      0.75      0.74       154
weighted avg       0.77      0.76      0.76       154

